In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="Socratic-8B.Q4_K_M:latest", temperature=0.2, base_url="http://localhost:11434")

In [7]:
from langchain_core.messages import SystemMessage, HumanMessage

human_prompt = """The claim is A physical book detailing the contents of Hunter Biden's abandoned laptop is being sold for $50.
And the justification of the fact-check is This claim is true. On June 1, 2023, the nonprofit group Marco Polo, which was founded by Donald Trump
 White House aide Garrett Ziegler, uploaded a large selection of content from a laptop owned and abandoned by U.S. President Joe Biden's son Hunter. 
 He had left it at a Delaware computer repair shop in October 2019."""


response = llm.invoke(human_prompt)
print(response.content) 

What is the evidence that Marco Polo has published all the contents of the laptop?
